# Hull-White 1 Factor Model

Hull-White model was one of the first practical exogenous models that attempted to fit to the market interest rate term structures. The model is described as:


## Hull white Model Variations

$dr_t = \big(\theta(t) - a r(t)\big) dt + \sigma_t dW_t$

$\theta$  has t (time) dependence — the **Hull–White model**

$dr_t = \big(\theta(t) - a (t) r(t)\big) dt + \sigma_t dW_t$

$\theta$ and $\alpha$  are both time-dependent — the **extended Vasicek model**


where 
- $a$ is the mean reversion constant, 
- $\sigma$ is the volatility parameter. 
- The parameter $\theta(t)$ is chosen in order to fit the input term structure of interest rates.

What is the **"right" value for parameters $a$ and $\sigma$**? This is the question that we address by calibrating to market instruments.

In [19]:
import QuantLib as ql
from collections import namedtuple
import math
import calibration_helper

In [20]:
today = ql.Date(15, ql.February, 2002);
settlement= ql.Date(19, ql.February, 2002);
ql.Settings.instance().evaluationDate = today;
term_structure = ql.YieldTermStructureHandle(ql.FlatForward(settlement,0.04875825,ql.Actual365Fixed()))
index = ql.Euribor1Y(term_structure)

In [21]:
CalibrationData = namedtuple("CalibrationData", 
                             "start, length, volatility")
data = [CalibrationData(1, 5, 0.1148),
        CalibrationData(2, 4, 0.1108),
        CalibrationData(3, 3, 0.1070),
        CalibrationData(4, 2, 0.1021),
        CalibrationData(5, 1, 0.1000 )]

In [22]:
model = ql.HullWhite(term_structure);
engine = ql.JamshidianSwaptionEngine(model)
swaptions = calibration_helper.create_swaption_helpers(data, index, term_structure, engine)

optimization_method = ql.LevenbergMarquardt(1.0e-8,1.0e-8,1.0e-8)
end_criteria = ql.EndCriteria(10000, 100, 1e-6, 1e-8, 1e-8)
model.calibrate(swaptions, optimization_method, end_criteria)

a, sigma = model.params()
print("a = %6.5f, sigma = %6.5f" % (a, sigma))

a = 0.04642, sigma = 0.00580


In [23]:
calibration_helper.calibration_report(swaptions, data)

----------------------------------------------------------------------------------
    Model Price    Market Price     Implied Vol      Market Vol       Rel Error
----------------------------------------------------------------------------------
        0.00878         0.00949         0.10620         0.11480        -0.07485
        0.00967         0.01008         0.10629         0.11080        -0.04061
        0.00866         0.00872         0.10634         0.10700        -0.00614
        0.00649         0.00623         0.10644         0.10210         0.04237
        0.00354         0.00332         0.10661         0.10000         0.06582
----------------------------------------------------------------------------------
Cumulative Error :         0.11614
